# Recurrent Neural Networks

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from d2l import torch as d2l
import collections
import re

d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():
    """Load the time machine dataset into a list of text lines."""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

Not all data are independant from each other  
Thats's not the case for a lot of problems, such as stock prediction, weather forecast, natural language processing (NLP), video, audio conversation, etc.  

CNN are powerful to process spatial information, Recurrent Neural Networks (RNN) were designed to better handle sequential information

As text data are widely accessible, a lot of the focus on sequence to sequence model has been targeted towards text processing (NLP)

<center>
    <img src='data/stock.png' width="65%" style="margin-left:auto; margin-right:auto"/>
    <p style="font-size:14px;">Source: <a href='tradingview.com'>Tradingview</a></p>
</center>

To predict stock price at time step $t$ you need to consider the price from $x_1$ to $x_t-1$  
When we are trying to predict the next stock price, we are predicting based on all historical data
$$x_t \sim P(x_t \mid x_{t-1}, \ldots, x_1)$$

One problem with our current model is that the number of inputs $x_1,..x_t-1$ is variable from one input to another  
Worse, the input can increase during training as for this stock prediction example!
Of course we can't often compute all the $x_1,..x_t-1$ as it would lead to something too big to be computed (intractable)  
So we often limit our computation to the last $n$ input example $x_t-n,..x_t-1$


Models that predict series based on some inputs are called *auto-regressive models* as they perform regression on themselves  
One common strategy is to keep a memory vector, to allow the model to store information there. Such models are called *latent auto-regressive models*
<center>
    <img src='data/sequence-model.svg' width="65%" style="margin-left:auto; margin-right:auto"/>
    <p style="font-size:14px;">Source: <a href='d2l.ai'>D2L</a></p>
</center>

One way to train this, is to pass historical observations to predict the next observation given the ones up to right now  
This is pretty expensive as you have dependencies between each step

Depending on the task solved, different architecture are used

<center>
    <img src='http://karpathy.github.io/assets/rnn/diags.jpeg' width="65%" style="margin-left:auto; margin-right:auto"/>
    <p style="font-size:14px;">Source: <a href='http://karpathy.github.io/2015/05/21/rnn-effectiveness/'>Karpathy</a></p>
</center>

Each rectangle in the diagram represents a vector, and arrows represent functions (e.g. matrix multiplication).

* Input vectors are shown in red.
* Output vectors are shown in blue.
* Green vectors represent the RNN's state (more on this soon).

From left to right:

1. Vanilla mode of processing without RNN, from fixed-sized input to fixed-sized output (e.g. image classification).
2. Sequence output (e.g. image captioning takes an image and outputs a sentence of words).
3. Sequence input (e.g. sentiment analysis where a given sentence is classified as expressing positive or negative sentiment).
4. Sequence input and sequence output (e.g. Machine Translation: an RNN reads a sentence in English and then outputs a sentence in French).
5. Synced sequence input and output (e.g. video classification where we wish to label each frame of the video).

Notice that in every case there are no pre-specified constraints on the lengths of sequences because the recurrent transformation (green) is fixed and can be applied as many times as needed.

**Many to one example**:

In [2]:
import torch.nn as nn

class ManyToOneRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(ManyToOneRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True) # (batch, sequence_length, X)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # Forward propagate RNN
        out, h = self.rnn(x, h0)
        
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])
        return out

The many to many example follows

But to train, we need some training data  
One big application of auto-regressive model is natural language processing  

The text is preprocess by being tokenized, each words are assigned a number

In [3]:
lines = read_time_machine()
print(f'number of lines in the text: {len(lines)}\n')
print(lines[42])

number of lines in the text: 3221

so most people think but wait a moment can an instantaneous


In [4]:
def tokenize(lines):
    return [line.split() for line in lines]
tokens = tokenize(lines)
for i in range(20, 30):
    print(tokens[i])

[]
['you', 'must', 'follow', 'me', 'carefully', 'i', 'shall', 'have', 'to', 'controvert', 'one', 'or', 'two']
['ideas', 'that', 'are', 'almost', 'universally', 'accepted', 'the', 'geometry', 'for']
['instance', 'they', 'taught', 'you', 'at', 'school', 'is', 'founded', 'on', 'a', 'misconception']
[]
['is', 'not', 'that', 'rather', 'a', 'large', 'thing', 'to', 'expect', 'us', 'to', 'begin', 'upon']
['said', 'filby', 'an', 'argumentative', 'person', 'with', 'red', 'hair']
[]
['i', 'do', 'not', 'mean', 'to', 'ask', 'you', 'to', 'accept', 'anything', 'without', 'reasonable']
['ground', 'for', 'it', 'you', 'will', 'soon', 'admit', 'as', 'much', 'as', 'i', 'need', 'from', 'you', 'you']


However, assigning a number to each of the word is not always pertinant  
Some words appear very unfrequently, and so, to reduce the size of the dataset we often skip very unfrequent word

Also, some special tokens are reserved for things such as `<pad>` for padding, `<bos>` for beginning and `<eos>` for an end of sequence

In [5]:
import torchtext
v1 = torchtext.vocab.build_vocab_from_iterator(tokens)

3221lines [00:00, 598920.65lines/s]


In [6]:
v1['you'], v1['<unk>'], v1['<pad>']

(24, 0, 1)

**Note**: State-of-the-art models (GPT3 at the time I am writting these lines) do not tokenize this way. 
They employ a technique called [byte pair encoding (BPE)](https://en.wikipedia.org/wiki/Byte_pair_encoding) which allows them to handle out-of-vocabulary words that were not present in the training dataset. 
Reference implementation: https://github.com/openai/tiktoken. 
Using BPE tokenization can significantly improve the performance of language models, as it allows them to handle a larger and more diverse vocabulary.

The objective of a language model (i.e., a model used for natural language processing tasks) is to estimate the joint probability of a word sequence

$$P(x_1, x_2, \ldots, x_T).$$

with $x_1, x_2, \ldots, x_T$ a sequence of words of length $T$ forming a sentence

Once we obtain such model, we can easily generate text by prediction the conditional probability of the next work w.r.t to the beginning of the sentence

$$x_t \sim P(x_t \mid x_{t-1}, \ldots, x_1)$$


The joint probability of a word sequence is defined by:

$$P(x_1, x_2, \ldots, x_T) = \prod_{t=1}^T P(x_t  \mid  x_1, \ldots, x_{t-1}).$$

Example: $P(\text{deep}, \text{learning}, \text{is}, \text{fun}) =  P(\text{deep}) P(\text{learning}  \mid  \text{deep}) P(\text{is}  \mid  \text{deep}, \text{learning}) P(\text{fun}  \mid  \text{deep}, \text{learning}, \text{is}).$

This is a self-supervised training task.
You can collect millions of text and train a model to predict next word each time.

The problem with such approach is that the dependency $x_{t-1}, \ldots, x_1$ can be very long  
To allow models to store information, as we discuss previously, we often use *auto-regressive* models that store in their hidden state $h_{t-1}$ the sequence of information up to $t-1$ they need to predict at time-step $t$

$$P(x_t \mid x_{t-1}, \ldots, x_1) \approx P(x_t \mid h_{t-1})$$

with $h_t$ defined recursively:
$h_t = f(x_{t}, h_{t-1})$

<center>
    <img src='data/rnn.svg' width="65%" style="margin-left:auto; margin-right:auto"/>
    <p style="font-size:14px;">Source: <a href='d2l.ai'>D2L</a></p>
</center>

Pytorch contains pre-defined RNN layers

In [7]:
num_hiddens = 256
rnn_layer = nn.RNN(len(v1), num_hiddens, num_layers=2, nonlinearity='tanh')

### By default, `nn.RNN` takes input of the form *(step, batch, input)*, if you want to pass the batch first, you have to specifiy it when building the RNN models

In [8]:
nn.RNN(len(v1), num_hiddens, num_layers=2, nonlinearity='tanh', batch_first=True)

RNN(4581, 256, num_layers=2, batch_first=True)

Recurrent layers are a bit special, in the sense that they don't just ouput the result of their operation but also their current state, which will be used in the next iteration.

In [9]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        # Trick to get the device of the model, see: https://stackoverflow.com/questions/58926054/how-to-get-the-device-type-of-a-pytorch-module-conveniently
        self.dummy_param = nn.Parameter(torch.empty(0))

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, batch_size=1):
        # for iter 0 we need an initial state
        return torch.zeros((self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                 device=self.dummy_param.device)
    
net = RNNModel(rnn_layer, vocab_size=len(v1))

While predicting the next word is best for accuracy, it makes the model complex as the dimensionality of the output is huge (i.e., the number of words in the vocabulary)  
To simplify things here, we will predict next character, making the problem easier.

In [10]:
# provides iterator and vocab
batch_size = 1
num_steps = 18 #maximum sequence length
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)
rnn_layer = nn.RNN(len(vocab), num_hiddens)
net = RNNModel(rnn_layer, vocab_size=len(vocab))

`D2L` defines their own `Vocab` class and doesn't use `torchtext` official `Vocab`

We first need to initialize the state of the `RNN`  
Usually we initialize the hidden state with a vector of zeros

In [11]:
state = net.begin_state(batch_size=batch_size)

The typical use case for `RNN` is to pass a *prefix* sentence and ask it to generate the rest of the sentence  
In that case, we usually *warm-start* the RNN by passing the prefix to get a *good* hidden state

In [12]:
prefix = 'time traveller '
outputs = [vocab[prefix[0]]]
for y in prefix[1:]:  # Warm-up period
    _, state = net(torch.LongTensor([outputs[-1]]).reshape(1, 1), state)
    outputs.append(vocab[y])
num_preds = 10
outputs = [vocab[prefix[0]]]
for _ in range(num_preds):  # Predict `num_preds` steps
    y, state = net(torch.LongTensor([outputs[-1]]).reshape(1, 1), state)
    outputs.append(int(y.argmax(dim=1).reshape(1)))
''.join([vocab.idx_to_token[i] for i in outputs])

'tgrpppppppp'

As we could have imagine, without training, the text generated is purely random

Trainning `RNN` is notoriously difficult, due to something called `Backpropagation Throught Time`  
As we need to backpropagate throught previous iteration, the gradient flow goes $T$ times trought the networks layers  
Leading to vanishing and exploding gradient problem, one common fix is to clip the norm of the gradient

In [13]:
torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)

tensor(0.)

Let's briefly train this network!

In [1]:
batch_size = 32
num_steps = 18 #maximum sequence length
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)
rnn_layer = nn.RNN(len(vocab), num_hiddens) # (seq, batch, X)
net = RNNModel(rnn_layer, vocab_size=len(vocab))

criteria = nn.CrossEntropyLoss()
optim = torch.optim.SGD(net.parameters(), lr=0.1)
state = net.begin_state(batch_size=batch_size)
num_epochs, num_steps = 1000, 18
for epoch in range(num_epochs):
    l_sum, n = 0.0, 0
    for X, Y in train_iter:
        state = net.begin_state(batch_size=batch_size)
        (output, state) = net(X, state)
        y = torch.transpose(Y, 0, 1).reshape(-1)
        l = criteria(output, y.long())
        optim.zero_grad()
        l.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)
        optim.step()
        l_sum += l.item() * y.numel()
        n += y.numel()
    if (epoch + 1) % 50 == 0:
        print(f'epoch {epoch + 1}, loss {l_sum / n:.4f}')

NameError: name 'd2l' is not defined

After training the output of the network should be a bit more coherent

In [16]:
with torch.no_grad():
    state = net.begin_state(batch_size=1)
    prefix = 'time traveller '
    outputs = [vocab[prefix[0]]]
    for y in prefix[1:]:  # Warm-up period
        _, state = net(torch.LongTensor([outputs[-1]]).reshape(1, 1), state)
        outputs.append(vocab[y])
    num_preds = 18
    outputs = [vocab[prefix[0]]]
    for _ in range(num_preds):  # Predict `num_preds` steps
        y, state = net(torch.LongTensor([outputs[-1]]).reshape(1, 1), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    print(''.join([vocab.idx_to_token[i] for i in outputs]))

the tranled the med
